In [145]:
import pandas as pd
from sklearn.utils import shuffle
import random as rand
import numpy as np

In [146]:
random = 24

In [212]:
data = pd.read_csv('games.csv')
data = data[["Metacritic", "Indie", "Presence", "Platform", 
             "Memory", "RatingsBreakdown", "ReleaseDate", "Soundtrack", 
             "OriginalCost", "DiscountedCost", "Controller",
             "Achievements", "Languages", "Storage", "Tags"]]

In [193]:
data.head()

,Metacritic,Indie,Presence,Platform,Memory,RatingsBreakdown,ReleaseDate,Soundtrack,OriginalCost,DiscountedCost,Controller,Achievements,Languages,Storage,Tags
0,83.0,0.0,1009588.0,"PC, Xbox 360, PlayStation 3",2 GB RAM,"recommended: 882, meh: 501, exceptional: 283, ...",2012-08-21,0.0,Free to Play,NaN,1.0,179.0,"English, Czech, Danish, Dutch, Finnish, French...",15 GB available space,"FPS, Shooter, Multiplayer, Competitive, Action..."
1,82.0,0.0,1007425.0,"PlayStation 5, Web, Xbox Series X, PC, Xbox On...",6 GB RAM,"recommended: 726, meh: 465, exceptional: 227, ...",2017-09-06,0.0,Free To Play,NaN,1.0,61.0,"English, French, Italian, German, Japanese, Ko...",105 GB available space,"Free to Play, Looter Shooter, FPS, Multiplayer..."
2,90.0,0.0,1009306.0,"Linux, macOS, PC",4 GB RAM,"meh: 496, recommended: 446, skip: 371, excepti...",2013-07-09,0.0,Free to Play,NaN,1.0,0.0,"English, Bulgarian, Czech, Danish, Dutch, Finn...",15 GB available space,"Free to Play, MOBA, Multiplayer, Strategy, eSp..."
3,71.0,0.0,1000781.0,PC,3 GB RAM,"recommended: 21, meh: 12, exceptional: 5, skip: 1",2014-04-04,0.0,$19.99,NaN,1.0,0.0,"English, French, German, Russian",85 GB available space,"RPG, MMORPG, Open World, Massively Multiplayer..."
4,68.0,0.0,777456.0,"PC, Xbox One",4 GB RAM,"recommended: 198, meh: 131, exceptional: 62, s...",2018-03-20,0.0,$39.99,NaN,1.0,308.0,"English, French, Italian, German, Russian",50 GB available space,"Adventure, Action, Pirates, Open World, Multip..."


In [159]:
rand.seed(random)
x = [y for y in range(len(data))]
x = rand.sample(x, 1000)
data = data.iloc[x]

In [160]:
data

,Metacritic,Indie,Presence,Platform,Storage,Memory,RatingsBreakdown,ReleaseDate,Soundtrack,OriginalCost,DiscountedCost,Controller,Achievements,Languages,Storage,Tags
23342,NaN,1.0,242.0,PC,500 MB available spac,NaN,NaN,2017-07-11,0.0,$0.99,NaN,1.0,0.0,English,500 MB available spac,"Action, Indie, Casual"
12545,65.0,1.0,1000648.0,PC,100 MB available spac,512 MB RAM,"meh: 7, exceptional: 1, recommended: 1",2011-02-04,0.0,$0.99,NaN,0.0,0.0,"English, Japanese",100 MB available spac,"Action, Indie, Puzzle, Cute, Mouse only"
27518,NaN,0.0,9.0,"Linux, macOS, PC",800 MB available space,4 GB RAM,NaN,2020-05-20,0.0,$2.50,NaN,0.0,0.0,English,800 MB available space,"Satire, RPG, Multiple Endings, Choices Matter,..."
19096,NaN,1.0,1625.0,"Linux, macOS, PC",200 MB available space,1 GB RAM,"meh: 4, skip: 3, exceptional: 1, recommended: 1",2014-02-13,0.0,$9.99,NaN,1.0,15.0,"English, Polish",200 MB available space,"Indie, Platformer, Adventure, Horror, Action, ..."
5983,NaN,1.0,236.0,"PC, macOS, iOS",NaN,2 GB RAM,"exceptional: 5, recommended: 4",2016-05-31,0.0,$9.99,NaN,0.0,27.0,English,NaN,"Roguelike Deckbuilder, Card Battler, Card Game..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11003,NaN,1.0,33.0,PC,170 MB available spac,2 GB RAM,NaN,2020-06-02,0.0,$0.99,NaN,0.0,0.0,English,170 MB available spac,"Indie, Casual, Strategy, Abstract, Puzzle, Min..."
21126,NaN,0.0,424.0,"Android, PC",250 MB available space,64 MB RAM,exceptional: 1,2003-05-01,0.0,$9.99,NaN,0.0,0.0,English,250 MB available space,"Strategy, Building, RTS, 4X, 2D, Historical, M..."
29047,NaN,1.0,327.0,PC,300 MB available space,2 GB RAM,NaN,2018-04-20,0.0,$3.99,NaN,1.0,0.0,"English, German, Portuguese, Russian",300 MB available space,"Indie, Action"
4750,NaN,1.0,95.0,"macOS, PC",1200 MB available spac,8 MB RAM,recommended: 2,NaN,0.0,Free,NaN,1.0,19.0,English,1200 MB available spac,"Free to Play, Open World, RPG, Indie, Simulati..."


In [259]:
len(data[data['Storage'].notna()]['Storage'].apply(lambda x: x.replace('GB', '')))


27491

In [235]:
data['Storage'][0].replace('GB', '')

'15  available space'